In [1]:
import pandas as pd
import sqlalchemy as sa
import urllib

In [2]:
params = urllib.parse.quote_plus('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=localhost;'
                      'Database=InteractDb_rc2;'
                      'Trusted_Connection=yes;')

engine = sa.create_engine("mssql+pyodbc:///?odbc_connect={}".format(params))
cnxn = engine.connect()


In [3]:
df_drug_ids = pd.read_sql(sa.text('SELECT $node_id, drugbank_id FROM drug_nodes'), cnxn)
df_drug_ids

,$node_id_C59E823CFB2442BFB04D7FE55B42561F,drugbank_id
0,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB00001
1,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB00002
2,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB00003
3,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB00004
4,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB00005
...,...,...
15230,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB17382
15231,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB17383
15232,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB17384
15233,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB17385


In [7]:
df_edges = pd.read_csv("../loader/ingest/drug_drug_edges.tsv", sep="\t")
df_edges=df_edges[["source","dst","desc"]]
df_edges

,source,dst,desc
0,DB00001,DB06605,Apixaban may increase the anticoagulant activi...
1,DB00001,DB06695,Dabigatran etexilate may increase the anticoag...
2,DB00001,DB01254,The risk or severity of bleeding and hemorrhag...
3,DB00001,DB01609,The risk or severity of gastrointestinal bleed...
4,DB00001,DB01586,The risk or severity of bleeding and bruising ...
...,...,...,...
2866926,DB17088,DB16694,The therapeutic efficacy of Famtozinameran can...
2866927,DB17088,DB16703,The therapeutic efficacy of Famtozinameran can...
2866928,DB17088,DB14681,The therapeutic efficacy of Famtozinameran can...
2866929,DB17088,DB14039,The risk or severity of adverse effects can be...


In [8]:
df = pd.merge(df_edges, df_drug_ids, left_on="source", right_on="drugbank_id", how="left")
df = pd.merge(df, df_drug_ids, left_on="dst", right_on="drugbank_id", how="left")
df.columns = ["src", "dst", "desc", "$from_id", "x1", "$to_id", "x2"]
df


,src,dst,desc,$from_id,x1,$to_id,x2
0,DB00001,DB06605,Apixaban may increase the anticoagulant activi...,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB00001,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB06605
1,DB00001,DB06695,Dabigatran etexilate may increase the anticoag...,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB00001,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB06695
2,DB00001,DB01254,The risk or severity of bleeding and hemorrhag...,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB00001,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB01254
3,DB00001,DB01609,The risk or severity of gastrointestinal bleed...,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB00001,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB01609
4,DB00001,DB01586,The risk or severity of bleeding and bruising ...,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB00001,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB01586
...,...,...,...,...,...,...,...
2866926,DB17088,DB16694,The therapeutic efficacy of Famtozinameran can...,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB17088,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB16694
2866927,DB17088,DB16703,The therapeutic efficacy of Famtozinameran can...,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB17088,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB16703
2866928,DB17088,DB14681,The therapeutic efficacy of Famtozinameran can...,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB17088,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB14681
2866929,DB17088,DB14039,The risk or severity of adverse effects can be...,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB17088,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB14039


In [9]:
df = df[["$from_id","$to_id","src","dst","desc"]]
df

,$from_id,$to_id,src,dst,desc
0,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB00001,DB06605,Apixaban may increase the anticoagulant activi...
1,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB00001,DB06695,Dabigatran etexilate may increase the anticoag...
2,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB00001,DB01254,The risk or severity of bleeding and hemorrhag...
3,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB00001,DB01609,The risk or severity of gastrointestinal bleed...
4,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB00001,DB01586,The risk or severity of bleeding and bruising ...
...,...,...,...,...,...
2866926,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB17088,DB16694,The therapeutic efficacy of Famtozinameran can...
2866927,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB17088,DB16703,The therapeutic efficacy of Famtozinameran can...
2866928,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB17088,DB14681,The therapeutic efficacy of Famtozinameran can...
2866929,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB17088,DB14039,The risk or severity of adverse effects can be...


In [10]:
df = df.dropna()

In [11]:
df.to_csv("./ingest/drug_drug_edges_ids.csv", sep='\t', index=False)